In [1]:
import multiprocessing
import pandas as pd
import numpy as np
import requests
import time
import os

In [2]:
dataset = pd.read_csv('../../../data/metadata/MIDI_with_Genre.csv')
save_path = '../../../lyrics/'
extension = ".txt"

In [2]:
midi = pd.read_csv('../../../data/metadata/MIDI_with_Genre.csv')
midi['msd_track_id'] = midi['track_id']
midi = midi.drop('track_id',axis=1).set_index('msd_track_id')
midi

,title,artist,genre
msd_track_id,,,
TRJJHFE128EF3518FD,I Don't Need,The Cranberries,rock
TRJJWLJ128F428AD88,Wild Angels,Martina McBride,country
TRJJXWI128F426BD83,Does Your Mother Know,Abba,pop
TRJNJWG128F92CFCB8,Psicopata,Capital Inicial,rock
TRJNRLV128E078D015,I Love The Way You Love Me (Remastered LP Ver...,John Michael Montgomery,country
...,...,...,...
TRBBQPP128F92EE611,Three Little Birds,Bob Marley & The Wailers,reggae
TRBBAEQ128F42B18B9,When You Love A Woman,Journey,rock
TRBBAIU128F92F3FB3,Bridge Over Troubled Water,Simon & Garfunkel,folk


In [3]:
dataset.head()

,Unnamed: 0,track_id,title,artist,genre
0,0,TRJJHFE128EF3518FD,I Don't Need,The Cranberries,rock
1,1,TRJJWLJ128F428AD88,Wild Angels,Martina McBride,country
2,2,TRJJXWI128F426BD83,Does Your Mother Know,Abba,pop
3,3,TRJNJWG128F92CFCB8,Psicopata,Capital Inicial,rock
4,4,TRJNRLV128E078D015,I Love The Way You Love Me (Remastered LP Ver...,John Michael Montgomery,country


In [4]:
# !pip install lyricsgenius

In [5]:
# library import and setting up the api auth code

import lyricsgenius
genius = lyricsgenius.Genius("z3kr--25YhkkXScS7S66RXqG-hVSrx4W2zWpTT6RFICAfavs7-sq1IhWrIuHd3Dc")


In [6]:
found_lyric = []
def lyrics_scrapper(song):
#     try:
    song_ = genius.search_song(song[1],song[2])
    if not song_: return
    lyric = song_.lyrics
    if lyric:
        print(f'saving to file {save_path}{song[0]}.txt...')
        with open(f'{save_path}{song[0]}.txt','w') as file:
            file.write(lyric)
        found_lyric.append(song[0])

In [7]:
data = dataset.iloc[:,1:4]
data = data.to_numpy().tolist()

In [8]:
# data[::-1]

In [3]:
pool = multiprocessing.Pool() 
pool = multiprocessing.Pool(processes=os.cpu_count())
pool.map(lyrics_scrapper, data)

# merge lyrics and midi metadata

In [ ]:
lyrics = os.listdir('../../../data/text')
lyrics = [lyric[:-4] for lyric in lyrics]
lyrics.sort()
lyrics = lyrics[1:]

In [ ]:
lyrics_df = pd.DataFrame(np.array([lyrics,lyrics]).T,columns=['index','msd_track_id']).set_index('index')
lyrics_df

In [ ]:
lyrics_midi_data = lyrics_df.join(midi,how='inner')
lyrics_midi_data.to_csv('../../../data/metadata/lyrics_midi_data.csv',index=False)

In [ ]:
lyrics_midi_data.groupby('genre').agg({'msd_track_id':'count'}).to_csv('../../../data/metadata/genrewise_data.csv')